# Introduction

This notebook continues the discussion of binary classification. In particular, In particular, we will look at applying logistic regression to the scikit-learn breast cancer toy data set. <br>
People looking at this notebook would greatly benefit from first brushing up on linear regression.

# Some Theory for Logistic Regression

## Why use scary logistic regression over the powerful and loved linear regression?

This is a reasonable question and merits some discussion before we dive into the mathematics of logistic regression. From a practical perspective, linear regression produces contnous output rather than probabilistic output, requiring extra effort to transform the answer into a particular class. While the output of linear regression is continuous, logistic regression operates in probability space, which is highly desirable when we want to classify a given sample.

# Building a Logistic Regression Model for Binary Classification

## Exploring the Toy Cancer Data

The medical sciences are becoming increasingly interested in data science and machine learning to build predictive models and find novel insights. We will first perform some preliminary exploration of the scikit-learn breast cancer toy dataset. This dataset is perfect for didactic purposes because the data comes the biopsies have only two classes: malignant or benign. <br>
According to the documentation, the data "features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass" (https://scikit-learn.org/dev/datasets/toy_dataset.html#breast-cancer-wisconsin-diagnostic-dataset). <br>
We will first load the data and targets into pandas DataFrames and take a brief look at the data.

In [4]:
import pandas as pd
from sklearn.datasets import load_breast_cancer

#unpack the data into separate data and target numpy arrays
#setting return_X_y argument to True puts the data and the targets 
#into different numpy arrays
data_array, target_array = load_breast_cancer(return_X_y = True)

In [5]:
#covariate names
column_headers = [
    "radius (mean)",
    "texture (mean)",
    "perimeter (mean)",
    "area (mean)",
    "smoothness (mean)",
    "compactness (mean)",
    "concavity (mean)",
    "concave points (mean)",
    "symmetry (mean)",
    "fractal dimension (mean)",
    "radius (standard error)",
    "texture (standard error)",
    "perimeter (standard error)",
    "area (standard error)",
    "smoothness (standard error)",
    "compactness (standard error)",
    "concavity (standard error)",
    "concave points (standard error)",
    "symmetry (standard error)",
    "fractal dimension (standard error)",
    "radius (worst)",
    "texture (worst)",
    "perimeter (worst)",
    "area (worst)",
    "smoothness (worst)",
    "compactness (worst)",
    "concavity (worst)",
    "concave points (worst)",
    "symmetry (worst)",
    "fractal dimension (worst)"
]

data = pd.DataFrame(data_array, columns = column_headers)
targets = pd.DataFrame(target_array, columns = ["cancer class"])
#include the cancer class in the data dataframe for filtering ppurposes
data = pd.concat([data, targets], axis = 1, sort = False)

In [6]:
#print some metadata
print("Shape of data:\t\t\t", data_array.shape)
print("Shape of target:\t\t", target_array.shape)
print("Number of benign cases:\t\t", data[data['cancer class'] == 1]['cancer class'].count())
print("Number of malignant cases:\t", data[data['cancer class'] == 0]['cancer class'].count())

Shape of data:			 (569, 30)
Shape of target:		 (569,)
Number of benign cases:		 357
Number of malignant cases:	 212


In [7]:
#set option so that we see can see all columns
pd.set_option('display.max_columns', None)
data.head()

,radius (mean),texture (mean),perimeter (mean),area (mean),smoothness (mean),compactness (mean),concavity (mean),concave points (mean),symmetry (mean),fractal dimension (mean),radius (standard error),texture (standard error),perimeter (standard error),area (standard error),smoothness (standard error),compactness (standard error),concavity (standard error),concave points (standard error),symmetry (standard error),fractal dimension (standard error),radius (worst),texture (worst),perimeter (worst),area (worst),smoothness (worst),compactness (worst),concavity (worst),concave points (worst),symmetry (worst),fractal dimension (worst),cancer class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [8]:
#0 is malignant
#1 is benign
targets.head()

,cancer class
0,0
1,0
2,0
3,0
4,0


For convenience and sanity, we will call a particular measurement, such as radius or compactness, an "attribute", and call a derived measurement, such as the mean compactness or worst perimeter, a "covariate". <br>
The above data has 10 different attributes and 30 covariates. Each particular attribute for a biopsy image has three related covariates: the mean value for the image, the "worst" value for the image and the standard error for the image. <br>
We will continue our initial data exploration by looking at summary statistics for the data overall and class-specific data.

In [9]:
data.describe()

,radius (mean),texture (mean),perimeter (mean),area (mean),smoothness (mean),compactness (mean),concavity (mean),concave points (mean),symmetry (mean),fractal dimension (mean),radius (standard error),texture (standard error),perimeter (standard error),area (standard error),smoothness (standard error),compactness (standard error),concavity (standard error),concave points (standard error),symmetry (standard error),fractal dimension (standard error),radius (worst),texture (worst),perimeter (worst),area (worst),smoothness (worst),compactness (worst),concavity (worst),concave points (worst),symmetry (worst),fractal dimension (worst),cancer class
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,0.405172,1.216853,2.866059,40.337079,0.007041,0.025478,0.031894,0.011796,0.020542,0.003795,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,0.277313,0.551648,2.021855,45.491006,0.003003,0.017908,0.030186,0.006170,0.008266,0.002646,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,0.111500,0.360200,0.757000,6.802000,0.001713,0.002252,0.000000,0.000000,0.007882,0.000895,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,0.232400,0.833900,1.606000,17.850000,0.005169,0.013080,0.015090,0.007638,0.015160,0.002248,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,0.324200,1.108000,2.287000,24.530000,0.006380,0.020450,0.025890,0.010930,0.018730,0.003187,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,0.478900,1.474000,3.357000,45.190000,0.008146,0.032450,0.042050,0.014710,0.023480,0.004558,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,2.873000,4.885000,21.980000,542.200000,0.031130,0.135400,0.396000,0.052790,0.078950,0.029840,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,1.000000


In [10]:
data[data['cancer class'] == 0].describe()

,radius (mean),texture (mean),perimeter (mean),area (mean),smoothness (mean),compactness (mean),concavity (mean),concave points (mean),symmetry (mean),fractal dimension (mean),radius (standard error),texture (standard error),perimeter (standard error),area (standard error),smoothness (standard error),compactness (standard error),concavity (standard error),concave points (standard error),symmetry (standard error),fractal dimension (standard error),radius (worst),texture (worst),perimeter (worst),area (worst),smoothness (worst),compactness (worst),concavity (worst),concave points (worst),symmetry (worst),fractal dimension (worst),cancer class
count,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.0
mean,17.462830,21.604906,115.365377,978.376415,0.102898,0.145188,0.160775,0.087990,0.192909,0.062680,0.609083,1.210915,4.323929,72.672406,0.006780,0.032281,0.041824,0.015060,0.020472,0.004062,21.134811,29.318208,141.370330,1422.286321,0.144845,0.374824,0.450606,0.182237,0.323468,0.091530,0.0
std,3.203971,3.779470,21.854653,367.937978,0.012608,0.053987,0.075019,0.034374,0.027638,0.007573,0.345039,0.483178,2.568546,61.355268,0.002890,0.018387,0.021603,0.005517,0.010065,0.002041,4.283569,5.434804,29.457055,597.967743,0.021870,0.170372,0.181507,0.046308,0.074685,0.021553,0.0
min,10.950000,10.380000,71.900000,361.600000,0.073710,0.046050,0.023980,0.020310,0.130800,0.049960,0.193800,0.362100,1.334000,13.990000,0.002667,0.008422,0.011010,0.005174,0.007882,0.001087,12.840000,16.670000,85.100000,508.100000,0.088220,0.051310,0.023980,0.028990,0.156500,0.055040,0.0
25%,15.075000,19.327500,98.745000,705.300000,0.094010,0.109600,0.109525,0.064620,0.174050,0.056598,0.390375,0.892825,2.715500,35.762500,0.005085,0.019662,0.026998,0.011415,0.014615,0.002688,17.730000,25.782500,119.325000,970.300000,0.130475,0.244475,0.326425,0.152750,0.276500,0.076302,0.0
50%,17.325000,21.460000,114.200000,932.000000,0.102200,0.132350,0.151350,0.086280,0.189900,0.061575,0.547200,1.102500,3.679500,58.455000,0.006209,0.028590,0.037125,0.014205,0.017700,0.003739,20.590000,28.945000,138.000000,1303.000000,0.143450,0.356350,0.404900,0.182000,0.310300,0.087600,0.0
75%,19.590000,23.765000,129.925000,1203.750000,0.110925,0.172400,0.203050,0.103175,0.209850,0.067075,0.757300,1.429250,5.206250,94.000000,0.007971,0.038910,0.050443,0.017497,0.022132,0.004892,23.807500,32.690000,159.800000,1712.750000,0.155975,0.447850,0.556175,0.210675,0.359225,0.102625,0.0
max,28.110000,39.280000,188.500000,2501.000000,0.144700,0.345400,0.426800,0.201200,0.304000,0.097440,2.873000,3.568000,21.980000,542.200000,0.031130,0.135400,0.143800,0.040900,0.078950,0.012840,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.170000,0.291000,0.663800,0.207500,0.0


In [11]:
data[data['cancer class'] == 1].describe()

,radius (mean),texture (mean),perimeter (mean),area (mean),smoothness (mean),compactness (mean),concavity (mean),concave points (mean),symmetry (mean),fractal dimension (mean),radius (standard error),texture (standard error),perimeter (standard error),area (standard error),smoothness (standard error),compactness (standard error),concavity (standard error),concave points (standard error),symmetry (standard error),fractal dimension (standard error),radius (worst),texture (worst),perimeter (worst),area (worst),smoothness (worst),compactness (worst),concavity (worst),concave points (worst),symmetry (worst),fractal dimension (worst),cancer class
count,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.00000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.000000,357.0
mean,12.146524,17.914762,78.075406,462.790196,0.092478,0.080085,0.046058,0.025717,0.174186,0.062867,0.284082,1.22038,2.000321,21.135148,0.007196,0.021438,0.025997,0.009858,0.020584,0.003636,13.379801,23.515070,87.005938,558.899440,0.124959,0.182673,0.166238,0.074444,0.270246,0.079442,1.0
std,1.780512,3.995125,11.807438,134.287118,0.013446,0.033750,0.043442,0.015909,0.024807,0.006747,0.112570,0.58918,0.771169,8.843472,0.003061,0.016352,0.032918,0.005709,0.006999,0.002938,1.981368,5.493955,13.527091,163.601424,0.020013,0.092180,0.140368,0.035797,0.041745,0.013804,0.0
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.051850,0.111500,0.36020,0.757000,6.802000,0.001713,0.002252,0.000000,0.000000,0.009539,0.000895,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156600,0.055210,1.0
25%,11.080000,15.150000,70.870000,378.200000,0.083060,0.055620,0.020310,0.015020,0.158000,0.058530,0.207300,0.79590,1.445000,15.260000,0.005212,0.011320,0.010990,0.006433,0.015600,0.002074,12.080000,19.580000,78.270000,447.100000,0.110400,0.112000,0.077080,0.051040,0.240600,0.070090,1.0
50%,12.200000,17.390000,78.180000,458.400000,0.090760,0.075290,0.037090,0.023440,0.171400,0.061540,0.257500,1.10800,1.851000,19.630000,0.006530,0.016310,0.018400,0.009061,0.019090,0.002808,13.350000,22.820000,86.920000,547.400000,0.125400,0.169800,0.141200,0.074310,0.268700,0.077120,1.0
75%,13.370000,19.760000,86.100000,551.100000,0.100700,0.097550,0.059990,0.032510,0.189000,0.065760,0.341600,1.49200,2.388000,25.030000,0.008534,0.025890,0.030560,0.011870,0.024060,0.004174,14.800000,26.510000,96.590000,670.000000,0.137600,0.230200,0.221600,0.097490,0.298300,0.085410,1.0
max,17.850000,33.810000,114.600000,992.100000,0.163400,0.223900,0.410800,0.085340,0.274300,0.095750,0.881100,4.88500,5.118000,77.110000,0.021770,0.106400,0.396000,0.052790,0.061460,0.029840,19.820000,41.780000,127.100000,1210.000000,0.200600,0.584900,1.252000,0.175000,0.422800,0.148600,1.0


In [12]:
#are summary statistics greater for the malignant class
data[data['cancer class'] == 0].describe() > data[data['cancer class'] == 1].describe()

,radius (mean),texture (mean),perimeter (mean),area (mean),smoothness (mean),compactness (mean),concavity (mean),concave points (mean),symmetry (mean),fractal dimension (mean),radius (standard error),texture (standard error),perimeter (standard error),area (standard error),smoothness (standard error),compactness (standard error),concavity (standard error),concave points (standard error),symmetry (standard error),fractal dimension (standard error),radius (worst),texture (worst),perimeter (worst),area (worst),smoothness (worst),compactness (worst),concavity (worst),concave points (worst),symmetry (worst),fractal dimension (worst),cancer class
count,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
mean,True,True,True,True,True,True,True,True,True,False,True,False,True,True,False,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,False
std,True,False,True,True,False,True,True,True,True,True,True,False,True,True,False,True,False,False,True,False,True,False,True,True,True,True,True,True,True,True,False
min,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,False,False,False
25%,True,True,True,True,True,True,True,True,True,False,True,True,True,True,False,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,False
50%,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,False
75%,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,False
max,True,True,True,True,False,True,True,True,True,True,True,False,True,True,True,True,False,False,True,False,True,True,True,True,True,True,False,True,True,True,False


We will now examine the ranges of the covariates

In [13]:
malignant_range = data[data['cancer class'] == 0].max() - data[data['cancer class'] == 0].min()
malignant_range

radius (mean)                           17.160000
texture (mean)                          28.900000
perimeter (mean)                       116.600000
area (mean)                           2139.400000
smoothness (mean)                        0.070990
compactness (mean)                       0.299350
concavity (mean)                         0.402820
concave points (mean)                    0.180890
symmetry (mean)                          0.173200
fractal dimension (mean)                 0.047480
radius (standard error)                  2.679200
texture (standard error)                 3.205900
perimeter (standard error)              20.646000
area (standard error)                  528.210000
smoothness (standard error)              0.028463
compactness (standard error)             0.126978
concavity (standard error)               0.132790
concave points (standard error)          0.035726
symmetry (standard error)                0.071068
fractal dimension (standard error)       0.011753


In [14]:
benign_range = data[data['cancer class'] == 1].max() - data[data['cancer class'] == 1].min()
benign_range

radius (mean)                           10.869000
texture (mean)                          24.100000
perimeter (mean)                        70.810000
area (mean)                            848.600000
smoothness (mean)                        0.110770
compactness (mean)                       0.204520
concavity (mean)                         0.410800
concave points (mean)                    0.085340
symmetry (mean)                          0.168300
fractal dimension (mean)                 0.043900
radius (standard error)                  0.769600
texture (standard error)                 4.524800
perimeter (standard error)               4.361000
area (standard error)                   70.308000
smoothness (standard error)              0.020057
compactness (standard error)             0.104148
concavity (standard error)               0.396000
concave points (standard error)          0.052790
symmetry (standard error)                0.051921
fractal dimension (standard error)       0.028945


In [15]:
malignant_range > benign_range

radius (mean)                          True
texture (mean)                         True
perimeter (mean)                       True
area (mean)                            True
smoothness (mean)                     False
compactness (mean)                     True
concavity (mean)                      False
concave points (mean)                  True
symmetry (mean)                        True
fractal dimension (mean)               True
radius (standard error)                True
texture (standard error)              False
perimeter (standard error)             True
area (standard error)                  True
smoothness (standard error)            True
compactness (standard error)           True
concavity (standard error)            False
concave points (standard error)       False
symmetry (standard error)              True
fractal dimension (standard error)    False
radius (worst)                         True
texture (worst)                        True
perimeter (worst)               

The first thing we may notice is the mean for all mean covariates except mean fractal dimension is larger for malignant samples than those for benign samples. This agrees with the model of cancer as abnormal cells that do not respect cellular signaling that limits normal somatic cell growth. Moreover, the range for malignant cells for mean radius, mean texture, mean perimeter, mean area, and mean compactness are all greater, which corresponds with the expected unregulated growth of cancer cells. <br>
Perhaps unsurprisingly, the mean for all worst covariates was greater for malignant compared to cancer cells. Furthermore, the range for all worst covariates except concavity is greater for maligcant cancer cells. This helps build some intuition that the above metrics may provide some discriminatory power for classifying images as cancer. <br>
We will focus in the next section on creating our model and measuring its effectivenesss with the metrics we outlined in pervious sections.